In [1]:
import os
import numpy
import matplotlib.pyplot as plt
import csv
import math
from operator import add

In [3]:
folderpath = 'C:/Users/Avinash Sookram/Documents/Tsys_Files'

OBS_TO_RERUN = [] #If there are any invalid observations, add names/RA_DEC to this list.

MJD = [] #MJD of obs
CRYOTEMP = [] #Temp of Cryo 
BOXTEMP = [] 
RA_DEC = [] #RA and DEC
AZIM = [] #Azimuth
ELEV = [] #Elevation
TSYS1_XL1 = []
TSYS1_YR1 = []
TSYS2_XL1 = []
TSYS2_YR1 = []
GAINS1_XL1 = []
GAINS1_YR1 = []
GAINS2_XL1 = []
GAINS2_YR1 = []


for filename in os.listdir(folderpath):
    
    f = os.path.join(folderpath, filename)
    
    with open(f, "r") as f:
        lines = f.readlines()
    
        if 'spect' in filename: #Takes care of regular spectral runs that were conducted.
            OBS_TO_RERUN.append(filename)

        else: 
            for line in lines: #Fix bug where when TSYS over 100, doesn't store
                if line.startswith("#"):
                    if "MJD" in line:
                        MJD.append(line.split("=")[1].strip())
                    elif "CRYOTEMP" in line:
                        CRYOTEMP.append(line.split("=")[1].strip())
                    elif "BOXTEMP" in line:
                        BOXTEMP.append(line.split("=")[1].strip())
                    elif "AZIM(deg)" in line:
                        AZIM.append(line.split("=")[1].strip())
                    elif "ELEV(deg)" in line:
                        ELEV.append(line.split("=")[1].strip())
                    elif "RA,DEC" in line:
                        RA_DEC.append(line.split("=")[1].strip())
                        
                    elif "TSYS1" in line:
                        Unclean_TSYS1= line.split('=')[1].strip() #get rid of # in beginning 
                        Space_TSYS1 = Unclean_TSYS1.split(' ') #separate values with ' ' instead, having numbers and ' '
                        no_whitespace_TSYS1 = [ele for ele in Space_TSYS1 if ele.strip()] #get rid of ' '
                        XL1, YL1 = no_whitespace_TSYS1[0], no_whitespace_TSYS1[1]
                        #print(XL1, YL1)
                        TSYS1_XL1.append(float(XL1)) #can parse values more reliably
                        TSYS1_YR1.append(float(YL1))
                        
                    elif "TSYS2" in line:
                        Unclean_TSYS2= line.split('=')[1].strip() #get rid of # in beginning 
                        Space_TSYS2 = Unclean_TSYS2.split(' ') #separate values with ' ' instead, having numbers and ' '
                        no_whitespace_TSYS2 = [ele for ele in Space_TSYS2 if ele.strip()] #get rid of ' '
                        XL1_2, YL1_2 = no_whitespace_TSYS2[0], no_whitespace_TSYS2[1]
                        #print(YL1_2)
                        TSYS2_XL1.append(float(XL1_2)) #can parse values more reliably
                        TSYS2_YR1.append(float(YL1_2))
                        
                    elif "GAIN1" in line: 
                        gain1_val = line.split("=")[1].strip()
                        gains1 = gain1_val.split(" ")
                        GAINS1_XL1.append(float(gains1[0]))
                        GAINS1_YR1.append(float(gains1[3]))
                        
                    elif "GAIN2" in line:
                        gain2_val = line.split("=")[1].strip()
                        gains2 = gain2_val.split(" ")
                        GAINS2_XL1.append(float(gains2[0]))
                        GAINS2_YR1.append(float(gains2[3])) 
                    
indices_to_delete = [] #There are some files where TSYS values are all 0... weird

for i, value in enumerate(TSYS1_XL1):
    if value == 0.0: #Make sure this isn't a string but a float value
        indices_to_delete.append(i) #gets indicies/observations where TSYS is 0.0
        
for index in sorted(indices_to_delete, reverse=True): #deletes observations that have the 0 TSYS values
    OBS_TO_RERUN.append(RA_DEC[index]) #Quickly add RA DEC to be rerun into rerun list
    del MJD[index]
    del CRYOTEMP[index]
    del BOXTEMP[index] 
    del AZIM[index]
    del ELEV[index]
    del RA_DEC[index]
    del TSYS1_XL1[index]
    del TSYS1_YR1[index]
    del TSYS2_XL1[index]
    del TSYS2_YR1[index]
    del GAINS1_XL1[index]
    del GAINS1_YR1[index]
    del GAINS2_XL1[index]
    del GAINS2_YR1[index]
    
#Convert elements of all the lists to float numbers
#Extra calculations
Elevation_To_Radians = list(map(lambda x: float(x)*numpy.pi/180, ELEV)) #Perform Elevation calibration conversions
SIN_Elevation = list(map(lambda x: math.sin(x), Elevation_To_Radians))
One_Div_SinEle = list(map(lambda x: 1/x, SIN_Elevation))

LN_Ele_Rads = list(map(lambda x: numpy.log(x), Elevation_To_Radians)) #Natural log of each Elevation calibration
LN_Sin_Ele = list(map(lambda x: numpy.log(x), SIN_Elevation))
LN_One_Div_SinEle = list(map(lambda x: numpy.log(x), One_Div_SinEle))

LN_TSYS1_XL1 = list(map(lambda x: numpy.log(x), TSYS1_XL1)) #Natural log of each TSYS
LN_TSYS1_YR1 = list(map(lambda x: numpy.log(x), TSYS1_YR1))
LN_TSYS2_XL1 = list(map(lambda x: numpy.log(x), TSYS2_XL1))
LN_TSYS2_YR1 = list(map(lambda x: numpy.log(x), TSYS2_YR1))

Sum_TSYS_XL1 = [sum(i) for i in zip(TSYS1_XL1, TSYS2_XL1)]
Sum_TSYS_YR1 = [sum(i) for i in zip(TSYS1_YR1, TSYS2_YR1)]
Sum_All_Tsys = [sum(i) for i in zip(TSYS1_XL1, TSYS2_XL1, TSYS1_YR1, TSYS2_YR1)]

Sum_GAINS_XL1 = [sum(i) for i in zip(GAINS1_XL1, GAINS2_XL1)]
Sum_GAINS_YR1 = [sum(i) for i in zip(GAINS1_YR1, GAINS2_YR1)]
Sum_All_Gains = [sum(i) for i in zip(GAINS1_XL1, GAINS2_XL1, GAINS1_YR1, GAINS2_YR1)]


Average_TSYS_XL1 = [Sum_TSYS_XL1[i] / Sum_GAINS_XL1[i] for i in range(len(Sum_TSYS_XL1))]
Average_TSYS_YR1 = [Sum_TSYS_YR1[i] / Sum_GAINS_YR1[i] for i in range(len(Sum_TSYS_YR1))]
Average_TSYS_ALL = [Sum_All_Tsys[i] / Sum_All_Gains[i] for i in range(len(Sum_All_Tsys))]



# Open a new .csv file for writing
with open('TSYS_Data.csv', 'w', newline='') as f:
    writer = csv.writer(f)

    # Write the lists to the .csv file as columns
    for row in zip(MJD, CRYOTEMP, BOXTEMP, RA_DEC, AZIM, ELEV, TSYS1_XL1, TSYS1_YR1, TSYS2_XL1, TSYS2_YR1, GAINS1_XL1, GAINS1_YR1, GAINS2_XL1, GAINS2_YR1, Elevation_To_Radians, SIN_Elevation, One_Div_SinEle, LN_Ele_Rads, LN_Sin_Ele, LN_One_Div_SinEle, LN_TSYS1_XL1, LN_TSYS1_YR1, LN_TSYS2_XL1, LN_TSYS2_YR1, Sum_TSYS_XL1, Sum_TSYS_YR1, Sum_All_Tsys, Sum_GAINS_XL1, Sum_GAINS_YR1, Sum_All_Gains, Average_TSYS_XL1, Average_TSYS_YR1, Average_TSYS_ALL):
        writer.writerow(row)


PermissionError: [Errno 13] Permission denied: 'TSYS_Data.csv'